# **Cat and Dog Classification using CNN**

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats/data

Create API token from Kaggle Account


In [1]:
# create directory

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
# download the dataset : Copy API command

!kaggle datasets download -d salader/dogs-vs-cats

 99% 1.05G/1.06G [00:09<00:00, 277MB/s]
100% 1.06G/1.06G [00:09<00:00, 116MB/s]


In [3]:
# unzip the dataset folder

import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
# import the libraries

import tensorflow as tf
import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [5]:
# generators- divide data into batches

train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [6]:
# Normalize - (0,255) --> (0,1)

def process(image,label):
  image = tf.cast(image/255, tf.float32)
  return image, label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)


# Model 1

In [7]:
# create CNN MODEL

model1 = Sequential()

model1.add(Conv2D(32, kernel_size=(3,3), padding = 'valid', activation = 'relu', input_shape = (256,256,3)))
model1.add(MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'valid'))

model1.add(Conv2D(64, kernel_size=(3,3), padding = 'valid', activation = 'relu'))
model1.add(MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'valid'))

model1.add(Conv2D(128, kernel_size=(3,3), padding = 'valid', activation = 'relu'))
model1.add(MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'valid'))

model1.add(Flatten())

model1.add(Dense(128, activation = 'relu'))
model1.add(Dense(64, activation = 'relu'))
model1.add(Dense(1, activation = 'sigmoid'))


In [8]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 128)       0

In [9]:
model1.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model1.fit(train_ds, epochs=10, validation_data = validation_ds)

Epoch 1/10
625/625 [==============================] - 65s 85ms/step - loss: 0.6545 - accuracy: 0.6161 - val_loss: 0.5830 - val_accuracy: 0.6838
Epoch 2/10
625/625 [==============================] - 59s 95ms/step - loss: 0.5039 - accuracy: 0.7514 - val_loss: 0.4585 - val_accuracy: 0.7860
Epoch 3/10
625/625 [==============================] - 52s 82ms/step - loss: 0.3993 - accuracy: 0.8167 - val_loss: 0.5223 - val_accuracy: 0.7894
Epoch 4/10
625/625 [==============================] - 55s 87ms/step - loss: 0.2759 - accuracy: 0.8824 - val_loss: 0.5239 - val_accuracy: 0.7934
Epoch 5/10
625/625 [==============================] - 54s 86ms/step - loss: 0.1528 - accuracy: 0.9393 - val_loss: 0.8190 - val_accuracy: 0.7892
Epoch 6/10
625/625 [==============================] - 52s 83ms/step - loss: 0.0912 - accuracy: 0.9667 - val_loss: 1.0178 - val_accuracy: 0.7768
Epoch 7/10
625/625 [==============================] - 51s 81ms/step - loss: 0.0639 - accuracy: 0.9779 - val_loss: 1.1038 - val_accuracy:

In [ ]:
import matplotlib.pyplot as plt

plt.title('Accuracy')
plt.plot(history.history['accuracy'], color='red', label = 'train')
plt.plot(history.history['val_accuracy'], color='blue', label = 'validation')
plt.legend()
plt.show()

# accuracy is increse but validation accuracy is less
# overfitting


In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], color='red', label = 'train')
plt.plot(history.history['val_loss'], color='blue', label = 'validation')
plt.legend()
plt.show()

# loss is decrese but validation loss is increase
# overfitting

In [ ]:
# prediction

In [ ]:
import cv2

test_img = cv2.imread('/content/cat.jpg')

plt.imshow(test_img)

In [ ]:
test_img.shape

In [ ]:
test_img = cv2.resize(test_img, (256,256))

In [ ]:
test_input = test_img.reshape(1, 256, 256, 3)

In [ ]:
model1.predict(test_input)

# Ways to Reduce Overfitting

  1 Add more data

  2 Data Auegumentation
  
  3 L1/ L2 regularization
  
  4 Dropout
  
  5 Batch Norm
  
  6 Reduce Complexity

# Model 2

In [ ]:
# model 2- Adding Batch Norm and Dropout

model2 = Sequential()

model2.add(Conv2D(64, kernel_size=(3,3), padding = 'valid', activation = 'relu', input_shape = (256,256,3)))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'valid'))

model2.add(Conv2D(64, kernel_size=(3,3), padding = 'valid', activation = 'relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'valid'))

model2.add(Conv2D(128, kernel_size=(3,3), padding = 'valid', activation = 'relu'))
model2.add(Conv2D(64, kernel_size=(3,3), padding = 'valid', activation = 'relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'valid'))

model2.add(Conv2D(256, kernel_size=(3,3), padding = 'valid', activation = 'relu'))
model2.add(Conv2D(64, kernel_size=(3,3), padding = 'valid', activation = 'relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'valid'))


model2.add(Flatten())

model2.add(Dense(128, activation = 'relu'))
model2.add(Dropout(0.2))
model2.add(Dense(64, activation = 'relu'))
model2.add(Dropout(0.2))
model2.add(Dense(32, activation = 'relu'))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation = 'sigmoid'))


In [ ]:
model2.summary()
model2.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [ ]:
history = model2.fit(train_ds, epochs=20, validation_data = validation_ds)

In [ ]:
import matplotlib.pyplot as plt

plt.title('Accuracy')
plt.plot(history.history['accuracy'], color='red', label = 'train')
plt.plot(history.history['val_accuracy'], color='blue', label = 'validation')
plt.legend()
plt.show()

plt.title('Loss')
plt.plot(history.history['loss'], color='red', label = 'train')
plt.plot(history.history['val_loss'], color='blue', label = 'validation')
plt.legend()
plt.show()



Overtting Occured.(best result on training data but not on test data)

# Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'content/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/content/train',  # target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/content/test',
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
# model 3

model3 = Sequential()

model3.add(Conv2D(32, kernel_size=(3,3), padding = 'valid', activation = 'relu', input_shape = (256,256,3)))
model3.add(BatchNormalization())
model3.add(MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'valid'))

model3.add(Conv2D(64, kernel_size=(3,3), padding = 'valid', activation = 'relu'))
model3.add(BatchNormalization())
model3.add(MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'valid'))

model3.add(Conv2D(128, kernel_size=(3,3), padding = 'valid', activation = 'relu'))
model3.add(Conv2D(64, kernel_size=(3,3), padding = 'valid', activation = 'relu'))
model3.add(BatchNormalization())
model3.add(MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'valid'))

model3.add(Conv2D(256, kernel_size=(3,3), padding = 'valid', activation = 'relu'))
model3.add(Conv2D(64, kernel_size=(3,3), padding = 'valid', activation = 'relu'))
model3.add(BatchNormalization())
model3.add(MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'valid'))

model3.add(Flatten())

model3.add(Dense(128, activation = 'relu'))
model3.add(Dropout(0.15))
model3.add(Dense(64, activation = 'relu'))
model3.add(Dropout(0.15))
model3.add(Dense(32, activation = 'relu'))
model3.add(Dropout(0.15))
model3.add(Dense(1, activation = 'sigmoid'))


In [ ]:
model3.summary()
model3.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [ ]:
# model 3

history = model3.fit(
        train_generator,
        steps_per_epoch=20000 // batch_size,
        epochs=25,
        validation_data=validation_generator,
        validation_steps=5000 // batch_size
        )

In [ ]:
import matplotlib.pyplot as plt

plt.title('Accuracy')
plt.plot(history.history['accuracy'], color='red', label = 'train')
plt.plot(history.history['val_accuracy'], color='blue', label = 'validation')
plt.legend()
plt.show()

plt.title('Loss')
plt.plot(history.history['loss'], color='red', label = 'train')
plt.plot(history.history['val_loss'], color='blue', label = 'validation')
plt.legend()
plt.show()



In [ ]:
model3.save_weights('model3.h5')  # always save your weights after training or during training

# **Conclusion**

  * Data Augmentation adds some noise in the data which act as a Regularization factor. Maybe this is the reason Validation accuracy is better than training accuracy.
  * So, compare to above 3 models, model 3 generalized model.

In [ ]:
import pickle

In [ ]:
pickle.dump(model3,open('model3.pkl', 'wb'))

# Transfer Learning

Now we'll try using transfer learning, VGG16

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
vgg = VGG16(
    weights = 'imagenet',
    include_top= False,
    input_shape=(256,256,3)
)


In [ ]:
# model 4

model4 = Sequential()

model4.add(vgg)

model4.add(Flatten())

model4.add(Dense(128, activation='relu'))
model4.add(Dense(64, activation='relu'))
model4.add(Dense(32, activation='relu'))
model4.add(Dense(1, activation='sigmoid'))

In [ ]:
model4.summary()

In [ ]:
model4.compile(optimizer='Adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# model 4

history = model4.fit(
        train_generator,
        steps_per_epoch=500,
        epochs=15,
        validation_data=validation_generator,
        validation_steps=5000 // batch_size
        )

In [ ]:
pickle.dump(model4,open('model4.pkl', 'wb'))